In [5]:
import imutils
from skimage.filters import threshold_local
import cv2
import base64
import os
import numpy as np
import random as rng
# from stackchain.widgets import rect2Box, shoWait, validDatedString
import pytesseract
from skimage import measure
import re
import datetime
import json

In [26]:

from PIL import Image, ImageEnhance, PngImagePlugin
import cv2
import numpy as np
from skimage import io
from skimage.transform import rotate
from skimage.color import rgb2gray
# from deskew import determine_skew
from matplotlib import pyplot as plt
from pdf2image import convert_from_path
import shutil
import glob
import os

def set_dpi(image, preproc_image):
    # im = Image.open(image)
    # im.save(preproc_image, dpi=(300,300))
    # return preproc_image
    dpi=(300,300)
    image = Image.open(image)
    
    # Create a new info dictionary to ensure dpi is set correctly
    info = PngImagePlugin.PngInfo()
    info.add_text("dpi", f"{dpi[0]} {dpi[1]}")
    
    # Save the image with the new DPI
    image.save(preproc_image, dpi=dpi, pnginfo=info)
    return preproc_image

def increase_contrast(image, preproc_image):
    im = Image.open(image)
    enhancer = ImageEnhance.Contrast(im)
    factor = 2 #increase contrast
    im_output = enhancer.enhance(factor)
    im_output.save(preproc_image)
    return preproc_image

def increase_sharpness(image, preproc_image):
    im = Image.open(image)
    enhancer = ImageEnhance.Sharpness(im)
    factor = 2
    im_output = enhancer.enhance(factor)
    im_output.save(preproc_image)
    return preproc_image

def convert_to_grayscale(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def convert_to_rgb(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def otsu_threshold(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    # im = cv2.threshold(im, 120, 255, cv2.THRESH_BINARY, cv2.THRESH_OTSU)[1]
    im = cv2.adaptiveThreshold(im,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def binary_threshold(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    ret, thresh1 = cv2.threshold(im, 120, 255, cv2.THRESH_BINARY)
    cv2.imwrite(preproc_image, thresh1)
    return preproc_image

def remove_noise(image, preproc_image):
    im = cv2.imread(image)
    # im = cv2.fastNlMeansDenoising(im, None, 20, 7, 21)
    im = cv2.fastNlMeansDenoisingColored(im, None, 10, 10, 7, 15)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def normalize_image(image, preproc_image):
    image = cv2.imread(image)
    norm_img = np.zeros((image.shape[0], image.shape[1]))
    img = cv2.normalize(image, norm_img, 0, 255, cv2.NORM_MINMAX)
    cv2.imwrite(preproc_image, img)
    return preproc_image

def image_scaling(image, preproc_image):

    im = Image.open(image)
    length_x, width_y = im.size
    factor = min(1, float(1024.0 / length_x))
    size = int(factor * length_x), int(factor * width_y)
    im_resized = im.resize(size, Image.Resampling.LANCZOS)
    # preproc_image = im_resized.NamedTemporaryFile(delete=False, suffix='.png')
    # preproc_image = preproc_image.name
    im_resized.save(preproc_image)
    return preproc_image

def preprocess_files():
    # print('hello')
    # combinations = ['convert_to_grayscale-set_dpi', 'increase_contrast-increase_sharpness-set_dpi', 'increase_contrast-convert_to_grayscale-set_dpi', 'convert_to_grayscale-remove_noise-set_dpi', 'increase_contrast-convert_to_grayscale-remove_noise-set_dpi',
                #    'convert_to_rgb-set_dpi']
    combinations = ['binary_threshold-set_dpi']
    # combinations = ['normalize_image-image_scaling-remove_noise-convert_to_grayscale']
    # combinations = ['set_dpi', 'normalize_image-remove_noise-convert_to_grayscale-set_dpi', 'convert_to_grayscale-set_dpi', 'increase_contrast-increase_sharpness-set_dpi', 'increase_contrast-convert_to_grayscale-set_dpi']
    psms = [3,4,6,11,12]
    for image in glob.glob('C:\\matsya\\india_dl\\lokesh_dl_original*'):
        # print('helloo')
        print("\n IMAGE --> ", image)
        # filename = os.path.split(image)[-1].split('.jpg')[0]
        # preproc_image = 'preprocess_images\\'+filename+'.jpg'
        # shutil.copy(image, preproc_image)
        for combination in combinations:
            if 'jpg' in image.lower():
                filename = os.path.split(image)[-1].split('.jpg')[0]
            elif 'png' in image.lower():
                filename = os.path.split(image)[-1].split('.png')[0]
            ext = image.split('.')[-1]
            preproc_image = 'C:\\matsya\\india_dl_preproc\\'+filename+'_'+combination+'.'+ext
            shutil.copy(image, preproc_image)
            methods = combination.split('-')
            for method in methods:
                if 'deskew' in method:
                    preproc_image = eval(method+'(preproc_image)')
                else:
                    preproc_image = eval(method+'(preproc_image, preproc_image)')

            print('preproc image---> ', preproc_image)
            im = Image.open(preproc_image)
            dpi = im.info.get('dpi', (72, 72))
            print('dpi---> ', dpi)
            # print(combination)
            # image_name = 'C:\\matsya\\india_dl_preproc\\' + filename+'_'+combination+'.'+ext
            # print(image_name)
            # img = cv2.imread(preproc_image)
            # cv2.imwrite(image_name, img)
            # im2 = Image.open(image_name)
            # new_dpi = im2.info.get('dpi', (72, 72))
            # print('new_dpi---> ', new_dpi)

            for psm in psms:
                parsed_output = 'C:\\matsya\\india_dl_preproc\\'+filename+'_'+str(psm)+'_'+combination
                command = 'tesseract {} {} -l eng+hin --psm {}'.format(preproc_image, parsed_output, psm)
                os.system(command)

# get_pdf_image()
preprocess_files()



 IMAGE -->  C:\matsya\india_dl\lokesh_dl_original.jpg
preproc image--->  C:\matsya\india_dl_preproc\lokesh_dl_original_binary_threshold-set_dpi.jpg
dpi--->  (300, 300)


In [15]:

from PIL import Image 
from pytesseract import pytesseract 
  
# Defining paths to tesseract.exe 
# and the image we would be using 
path_to_tesseract = r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"
image_path = 'C:\\matsya\\india_dl_preproc\\lokesh_dl_scanned_set_dpi.png'
  
# Opening the image & storing it in an image object 
img = Image.open(image_path) 
  
# Providing the tesseract executable 
# location to pytesseract library 
pytesseract.tesseract_cmd = path_to_tesseract 
  
# Passing the image object to image_to_string() function 
# This function will extract the text from the image 
text = pytesseract.image_to_string(img) 
  
# Displaying the extracted text 
print(text[:-1])

HR3578594
DLNUMBER HR2620170006785 Inv Carr No.
LOKESH YADAV
BHUPINDRA SINGH
40-Sep-1998 Validity (NT) 14-Apr-2037
Be
‘282 Tate Gurgaon HR 122001

uw See
news Shear

oa el
ped

lbouing Authorty $00 BADSHAYPUR

a C)

Ise Date zap a007
RE Sarto
pevek 2a


In [9]:
# image = 'C:\\matsya\\india_dl\\shruti_dl_scanned.png'
# im = Image.open(image)
# im.save('sample.png', dpi=(300,300))
# im.info.get('dpi')

from PIL import Image, PngImagePlugin

def get_dpi(image_path):
    image = Image.open(image_path)
    dpi = image.info.get('dpi', (72, 72))  # Default to (72, 72) if dpi info is not present
    return dpi

def change_dpi(image_path, output_path, dpi=(300, 300)):
    # Open the image
    image = Image.open(image_path)
    
    # Create a new info dictionary to ensure dpi is set correctly
    info = PngImagePlugin.PngInfo()
    info.add_text("dpi", f"{dpi[0]} {dpi[1]}")
    
    # Save the image with the new DPI
    image.save(output_path, dpi=dpi, pnginfo=info)
    
    print(f"Image saved at {output_path} with DPI {dpi}")

# Example usage
image_path = 'C:\\matsya\\india_dl_preproc\\deependra_dl_scanned_normalize_image-remove_noise-convert_to_grayscale-set_dpi.png'
output_path = 'sample.jpg'

# Print current DPI
current_dpi = get_dpi(image_path)
print(f"Current DPI: {current_dpi}")

# # Change DPI
# change_dpi(image_path, output_path, dpi=(300, 300))

# # Verify the new DPI
# new_dpi = get_dpi(output_path)
# print(f"New DPI: {new_dpi}")


Current DPI: (299.9994, 299.9994)


In [21]:
command = 'tesseract C:\matsya\india_dl\shruti_dl_original.jpg stdout --psm 4 -l eng'
print(os.popen(command).read())

a
oe ae

union oF nvia Driving Licence ®) ®

ware ae at Fee

tere eae. mag eM Ne. othe eg a

ni



In [11]:
W = 800
fontFace = cv2.FONT_HERSHEY_PLAIN
cleanText = r"[^A-Z0-9.,\-\s/]"

image = cv2.imread('C:\\matsya\\us_dl\\dl_6.jpg')

resize_proc = imutils.resize(image, width=W)
resize_orig = imutils.resize(image, width=W)

In [12]:
def cleanImage(image, stage=0):
    V = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    topHat = cv2.morphologyEx(V, cv2.MORPH_TOPHAT, kernel)
    blackHat = cv2.morphologyEx(V, cv2.MORPH_BLACKHAT, kernel)
    add = cv2.add(V, topHat)
    subtract = cv2.subtract(add, blackHat)
    if stage == 1:
        return subtract
    T = threshold_local(subtract, 29, offset=35, method="gaussian", mode="mirror")
    thresh = (subtract > T).astype("uint8")*255
    if stage == 2:
        return thresh
    thresh = cv2.bitwise_not(thresh)
    return thresh

In [13]:
def extractROIs(image, origin, minArea=1800, minHeight=25, minWidth=22):
    cnts = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    roisAsRects = []
    for c in cnts:
        rect = cv2.minAreaRect(c)
        (_, _), (rh, rw), _ = rect
        if rh>0:
            ratio = float(rw)/rh
            area = rw*rh
            if area>minArea and rh>minHeight and rw>minWidth and (ratio>1 or ratio<0.5):
                roisAsRects.append(rect)
    return roisAsRects

In [ ]:
def rect2box(x, y, w, h):
    x1 = r[0]
    y1 = r[1]
    x2 = r[0] + w
    y2 = r[1] + h
    return x1, y1, x2, y2

def cropRois(image, rects, multHeight=0.73, multWidth=0.97, topHeightCrop=30):
    crops = []
    data = {}
    angles = []
    for r in rects:
        box = rect2Box(r)
        

In [1]:
import image_to_scan

def scan_image(image_path):
    # Load the image
    image = image_to_scan.load_image(image_path)
    
    # Perform scanning operation (assuming a method exists)
    scanned_data = image_to_scan.scan(image)
    
    # Process and print the scanned data
    print(scanned_data)

# Example usage
scan_image('C:\matsya\dl_preprocess\dl_shruti.jpg')


AttributeError: module 'image_to_scan' has no attribute 'load_image'

In [13]:
import os

command = 'tesseract C:\matsya\india_dl\shruti_dl_scanned.png stdout --psm 4 -l eng'
print(os.popen(command).read())

be wvonorwoiaDriving Licence @) @




In [4]:
# Python program to illustrate 
# simple thresholding type on an image 
	
# organizing imports 
import cv2 
import numpy as np 

# path to input image is specified and 
# image is loaded with imread command 
img = cv2.imread('C:\\matsya\\india_dl\\shruti_dl_original.jpg') 

# cv2.cvtColor is applied over the 
# image input with applied parameters 
# to convert the image in grayscale 
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

# applying different thresholding 
# techniques on the input image 
# all pixels value above 120 will 
# be set to 255 
ret, thresh1 = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY) 
ret, thresh2 = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY_INV) 
ret, thresh3 = cv2.threshold(img, 120, 255, cv2.THRESH_TRUNC) 
ret, thresh4 = cv2.threshold(img, 120, 255, cv2.THRESH_TOZERO) 
ret, thresh5 = cv2.threshold(img, 120, 255, cv2.THRESH_TOZERO_INV) 

# # the window showing output images 
# # with the corresponding thresholding 
# # techniques applied to the input images 
cv2.imshow('Binary Threshold', thresh1) 
# cv2.imshow('Binary Threshold Inverted', thresh2) 
# cv2.imshow('Truncated Threshold', thresh3) 
# cv2.imshow('Set to 0', thresh4) 
# cv2.imshow('Set to 0 Inverted', thresh5) 
	
# # De-allocate any associated memory usage 
if cv2.waitKey(0) & 0xff == 27: 
	cv2.destroyAllWindows() 


In [34]:
!pip uninstall opencv-python

^C


In [2]:
import cv2
img = cv2.imread('C:\\matsya\\india_dl\\shruti_dl_original.jpg')
cv2.imshow('img', img)
cv2.waitKey(0)

-1

In [19]:
final_image = set_image_dpi('C:\\matsya\\india_dl\\shruti_dl_original.jpg')

UnboundLocalError: local variable 'temp_file' referenced before assignment